# EDA
Reference:
* https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition
* https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-crypto-forecasting
* https://www.kaggle.com/vbmokin/g-research-crypto-forecasting-baseline-fe
* https://www.kaggle.com/hyewon328/time-series-modeling-guide-with-cv-and-optuna
* https://www.kaggle.com/julian3833/g-research-starter-lgbm-pipeline

## Load the training set

In [1]:
import os
import time
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly import subplots

import seaborn as sns
cmap = sns.color_palette()

In [2]:
data_folder = "../input/g-research-crypto-forecasting/"
!ls $data_folder

In [3]:
crypto_df = pd.read_csv(data_folder + 'train.csv')

In [4]:
crypto_df.head()

We can see that each row of the data set has the trading data for an asset, at a given minute timestamp, described in detail below. 

## Data features

In [5]:
asset_details = pd.read_csv(data_folder + 'asset_details.csv')
asset_details.sort_values("Weight", ascending=False)

In [6]:
fig_assets_bar=px.bar(asset_details.sort_values("Weight"), x="Asset_Name" , y="Weight", color="Weight", title="Popular Cryptocurrency Weight Distribution")
fig_assets_bar.show()

In [7]:
fig_assets_pie = px.pie(asset_details, values='Weight', names='Asset_Name', title='Popular Cryptocurrency Weight Proportion')
fig_assets_pie.show()

## Start & End Time

 The data dating back to 2018. All the coins have the same end datetime at 2021.

In [8]:
asset_names_dict = {row["Asset_Name"]:row["Asset_ID"] for ind, row in asset_details.iterrows()}
asset_names_dict

In [9]:
time_list = []
for coin_name in asset_details["Asset_Name"]:
    coin_df = crypto_df[crypto_df["Asset_ID"]==asset_names_dict[coin_name]].set_index("timestamp")
    start_time = coin_df.index[0].astype('datetime64[s]')
    end_time = coin_df.index[-1].astype('datetime64[s]')
    time_list.append([coin_name, start_time, end_time])
time_df = pd.DataFrame(time_list)
time_df.columns = ["Asset_Name", "Start_Time", "End_Time"]
time_df

## Candlestick charts

In [10]:
def candelstick_chart(data,title):
    candlestick = go.Figure(data = [go.Candlestick(x =data.index, open = data[('Open')], high = data[('High')], low = data[('Low')], close = data[('Close')])])
    candlestick.update_xaxes(title_text = 'Time', rangeslider_visible = True)
    candlestick.update_layout(
    title = {
        'text': '{:} Candelstick Chart'.format(title),
        'y':0.90,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
    candlestick.update_yaxes(title_text = 'Price in USD', ticksuffix = '$')
    return candlestick

# plot candlestick charts for all the assets
for ix,coin_name in enumerate(asset_details["Asset_Name"]):
    coin_df = crypto_df[crypto_df["Asset_ID"]==asset_names_dict[coin_name]].set_index("timestamp")
    coin_df_mini = coin_df.iloc[-300:]
    fig = candelstick_chart(coin_df_mini,coin_name)
    fig.show()

In [11]:
# plot candlestick charts for all the assets
fig = subplots.make_subplots(rows=7,cols=2)
for ix,coin_name in enumerate(asset_details["Asset_Name"]):
    coin_df = crypto_df[crypto_df["Asset_ID"]==asset_names_dict[coin_name]].set_index("timestamp")
    coin_df_mini = coin_df.iloc[-300:]
    column = lambda ix: 1 if ix % 2 == 0 else 2
    candlestick = go.Candlestick(x=coin_df_mini.index, open=coin_df_mini['Open'], high=coin_df_mini['High'], low=coin_df_mini['Low'], close=coin_df_mini['Close'])
    fig = fig.add_trace(candlestick, row=((ix//2) + 1), col=column(ix))
    fig.update_xaxes(row=((ix//2) + 1), col=column(ix), title_text=coin_name+" - Time", rangeslider_visible=False)
    fig.update_yaxes(title_text='Price in USD', ticksuffix=' $')
fig.update_layout(title_text="Candlestick Charts", height=2100)
fig.show()

## Dealing with missing data


In [12]:
crypto_df.isna().sum().sort_values(ascending = False)

Let's see the missing data. There is a lot of missing data in the `Target` column.

In [13]:
missing_list = []
for coin_name in asset_details["Asset_Name"]:
    coin_df = crypto_df[crypto_df["Asset_ID"]==asset_names_dict[coin_name]].set_index("timestamp")
    missing_list.append([coin_name, coin_df.shape[0]] + coin_df.isna().sum().tolist())
missing_df = pd.DataFrame(missing_list)
missing_df.columns = ["Asset_Name", "TotalRows", 'Missing_Asset_ID', 'Missing_Count', 'Missing_Open', 
                      'Missing_High', 'Missing_Low', 'Missing_Close', 'Missing_Volume', 'Missing_VWAP', 'Missing_Target']
missing_df

Missing asset data, for a given minute, is not represented by NaN's, but instead by the absence of those rows. We can check the timestamp difference between consecutive rows to see if there is missing data. The data is collected once every 60 seconds. If we have a gap of more than one minute between consecutive rows, then it indicates that the data is missing in between.

In [14]:
eth = crypto_df[crypto_df["Asset_ID"]==6].set_index("timestamp") # Asset_ID = 6 for Ethereum
(eth.index[1:]-eth.index[:-1]).value_counts().head()

In [15]:
gap_list = []
for coin_name in asset_details["Asset_Name"]:
    coin_df = crypto_df[crypto_df["Asset_ID"]==asset_names_dict[coin_name]].set_index("timestamp")
    gap_df = (coin_df.index[1:]-coin_df.index[:-1]).value_counts().reset_index()
    n_gaps = gap_df[gap_df["index"]!=60]["timestamp"].sum()
    max_gap = gap_df["index"].max()
    gap_list.append([coin_name, n_gaps, max_gap])
gap_df = pd.DataFrame(gap_list)
gap_df.columns = ["Asset_Name", "number_of_gaps", "max_gap_in_seconds"]
gap_df

Notice that there are many gaps in the data. To work with most time series models, we should preprocess our data into a format without time gaps. To fill the gaps, we can use the `.reindex()` method for forward filling, filling gaps with the previous valid value. 

In [16]:
eth = eth.reindex(range(eth.index[0],eth.index[-1]+60,60),method='pad')

And check that are no time gaps now.

In [17]:
(eth.index[1:]-eth.index[:-1]).value_counts().head()

In [18]:
eth

## Close prices visualisation

In [19]:
f = plt.figure(figsize=(15,30))
for ix, coin_name in enumerate(asset_details["Asset_Name"]):
    coin_df = crypto_df[crypto_df["Asset_ID"]==asset_names_dict[coin_name]].set_index("timestamp") 
    coin_df = coin_df.reindex(range(coin_df.index[0],coin_df.index[-1]+60,60),method='pad') # fill missing values
    ax = f.add_subplot(7,2,ix+1)
    plt.plot(coin_df['Close'], label=coin_name, color=cmap[ix%10])
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel(coin_name+' Closing Price')
    plt.title(coin_name+' - Overall')
plt.tight_layout()
plt.show()

The assets have quite different history, but we could check if they correlate in recent times.

In [20]:
# auxiliary function, from datetime to timestamp
totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))

# create intervals
f = plt.figure(figsize=(15,30))
for ix, coin_name in enumerate(asset_details["Asset_Name"]):
    coin_df = crypto_df[crypto_df["Asset_ID"]==asset_names_dict[coin_name]].set_index("timestamp") 
    coin_df = coin_df.reindex(range(coin_df.index[0],coin_df.index[-1]+60,60),method='pad')  # fill missing values
    coin_df_mini = coin_df.loc[totimestamp('01/07/2021'):totimestamp('01/08/2021')]
    ax = f.add_subplot(7,2,ix+1)
    plt.plot(coin_df_mini['Close'], label=coin_name, color=cmap[ix%10])
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel(coin_name+' Closing Price')
    plt.title(coin_name+' - 1 Month Intervals')
plt.tight_layout()
plt.show()

On shorter intervals we can visually see some potential correlation, with some simultaneous ups and downs. A better format for analyzing such movements is by calculating asset returns. 

## Log returns

In order to analyze price changes for an asset we can deal with the price difference. However, different assets exhibit different price scales, so that the their returns are not readily comparable. We can solve this problem by computing the percentage change in price instead, also known as the return. This return coincides with the percentage change in our invested capital.

The `Target` column represents 15 minute residualized returns.

In [21]:
f = plt.figure(figsize=(15,30))
for ix, coin_name in enumerate(asset_details["Asset_Name"]):
    coin_df = crypto_df[crypto_df["Asset_ID"]==asset_names_dict[coin_name]].set_index("timestamp") 
    coin_df = coin_df.reindex(range(coin_df.index[0],coin_df.index[-1]+60,60),method='pad')  # fill missing values
    coin_df_mini = coin_df.loc[totimestamp('01/07/2021'):totimestamp('01/08/2021')]
    ax = f.add_subplot(7,2,ix+1)
    plt.plot(coin_df_mini['Target'], label=coin_name, color=cmap[ix%10])
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel(coin_name+' Residualized Return')
    plt.title(coin_name+' - 1 Month Residualized Return')
plt.tight_layout()
plt.show()

Returns are widely used in finance, however log returns are preferred for mathematical modelling of time series, as they are additive across time. Also, while regular returns cannot go below -100%, log returns are not bounded.

To compute the log return, we can simply take the logarithm of the ratio between two consecutive prices. The first row will have an empty return as the previous value is unknown, therefore the empty return data point will be dropped.

In [22]:
# define function to compute log returns
def log_return(series, periods=1):
    return np.log(series).diff(periods=periods)

We can visualize the log return for our assets. See how the signal now looks more like white noise, with less drift than the time series for prices.

In [23]:
f = plt.figure(figsize=(15,30))
for ix, coin_name in enumerate(asset_details["Asset_Name"]):
    coin_df = crypto_df[crypto_df["Asset_ID"]==asset_names_dict[coin_name]].set_index("timestamp") 
    coin_df = coin_df.reindex(range(coin_df.index[0],coin_df.index[-1]+60,60),method='pad')  # fill missing values
    coin_df_mini = coin_df.loc[totimestamp('01/07/2021'):totimestamp('01/08/2021')]
    logreturn = log_return(coin_df_mini['Close'].fillna(0))[1:] # drop the empty first row
    ax = f.add_subplot(7,2,ix+1)
    plt.plot(logreturn, label=coin_name, color=cmap[ix%10])
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel(coin_name+' Log Return')
    plt.title(coin_name+' - 1 Month Log Return')
plt.tight_layout()
plt.show()

## Correlation between assets

We hypothesized before that crypto asset returns may exhibit some correlation. Let's check this in more detail now.

Note the high but variable correlation between the assets. It would be critical for this time series challenge on how to perform forecasts in a highly non-stationary environment.

A stationary behaviour of a system or a process is characterized by non-changing statistical properties over time such as the mean, variance and autocorrelation. On the other hand, a non-stationary behaviour is characterized by a continuous change of statistical properties over time. Stationarity is important because many useful analytical tools and statistical tests and models rely on it.

We can also check the correlation between all assets visualizing the correlation matrix. Note how some assets have much higher pairwise correlation than others.

In [24]:
all_assets_df = pd.DataFrame([])
for coin_id, coin_name in zip(asset_details.Asset_ID, asset_details.Asset_Name):
    coin_df = crypto_df[crypto_df["Asset_ID"]==coin_id].set_index("timestamp")
    coin_df = coin_df.reindex(range(coin_df.index[0],coin_df.index[-1]+60,60),method='pad') # fill missing values
    coin_df_mini = coin_df.loc[totimestamp('01/07/2021'):totimestamp('01/08/2021')]
    return_values = log_return(coin_df_mini["Close"].fillna(0))[1:] # drop the empty first row
    return_values.name = coin_name
    all_assets_df = all_assets_df.join(return_values, how="outer")

plt.imshow(all_assets_df.corr());
plt.yticks(asset_details.Asset_ID.values, asset_details.Asset_Name.values);
plt.xticks(asset_details.Asset_ID.values, asset_details.Asset_Name.values, rotation='vertical');
plt.colorbar();

# Feature Engineering

In [25]:
# get new features
def get_features(data):
    df_feat = data[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    
    df_feat['Upper_Shadow'] = df_feat['High'] - np.maximum(df_feat['Close'], df_feat['Open'])
    df_feat['Lower_Shadow'] = np.minimum(df_feat['Close'], df_feat['Open']) - df_feat['Low']
    
    df_feat['High2lLow'] = df_feat['High'] / df_feat['Low']
    df_feat['Open2Close'] = df_feat['Close'] / df_feat['Open']
    df_feat['Volume2Count'] = df_feat['Volume'] / (df_feat['Count'] + 1)
    
    df_feat['Diff'] = df_feat['Close'] - df_feat['Open']
    df_feat['Avg_Volume'] = df_feat['Volume'] / df_feat['Count']
    df_feat['Avg_Price'] = (df_feat['Open'] + df_feat['High'] + df_feat['Low'] + df_feat['Close']) / 4
    
#     df_feat['MA_20'] = df_feat['Close'].rolling(window = 20, min_periods = 1).mean()
#     df_feat['MA_120'] = df_feat['Close'].rolling(window = 120, min_periods = 1).mean()
        
    return df_feat

In [26]:
# Get X and y
def get_data_for_asset(df_train, asset_id):
    df = df_train[df_train["Asset_ID"] == asset_id]    
    df_proc = get_features(df)
    df_proc['y'] = df['Target']
    
    df_proc = df_proc.fillna(0)
    
    X = df_proc.drop("y", axis=1)
    y = df_proc["y"]
    
    return X, y

# Model training and prediction

In [27]:
for asset_id in asset_details['Asset_ID']:
    currency = crypto_df[crypto_df["Asset_ID"]==asset_id] #.set_index("timestamp")
    currency = currency.reindex(range(currency.index[0],currency.index[-1]+60,60),method='pad')
    crypto_df[crypto_df["Asset_ID"]==asset_id] = currency

In [28]:
from lightgbm import LGBMRegressor

In [29]:
def model_training(LGBM_model,X,y):
    model = LGBM_model
    model.fit(X,y)
    return model

In [30]:
%%time
LGBM_model = LGBMRegressor(n_estimators = 5000,learning_rate=0.1,max_depth = 20,num_leaves = 700,
                      colsample_bytree = 0.8,subsample = 0.7,seed = 46)

Xs = {}
ys = {}
models = {}

for asset_id, asset_name in zip(asset_details['Asset_ID'], asset_details['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    X, y = get_data_for_asset(crypto_df, asset_id)    
    model = model_training(LGBM_model,X,y)
    Xs[asset_id], ys[asset_id], models[asset_id] = X, y, model

# Prediction and submission

In [31]:
import gresearch_crypto
env = gresearch_crypto.make_env()   
iter_test = env.iter_test()    

for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df['Target'] = 0  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions
    
for i, (df_test, df_pred) in enumerate(iter_test):
    for j, row in df_test.iterrows():
        try:
            model = models[row['Asset_ID']]
            x_test = get_features(row)
            y_pred = model.predict([x_test])[0]

            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
        
        except:
            print(f'{i}-th iteration of the test dataset, {j}-th row - there was the exception, then set Target = 0')
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
            
        # Print just one sample row to get a feeling of what it looks like        
        if i == 0 and j == 0:
            print('Example of the x_test data')
            display(x_test)

    # Display the first prediction dataframe
    if i == 0:
        print('Example of the prediction for test data')
        display(df_pred)
    df_pred['Target'] = df_pred['Target'].fillna(0)

    # Send submissions
    env.predict(df_pred)